# Models: Trees

In [1]:
import os
import gc
import warnings
import json

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
import seaborn as sns
import statsmodels as sm
from prettytable import PrettyTable

In [3]:
from sklearn.preprocessing import (
    OrdinalEncoder,
    StandardScaler,
    RobustScaler,
    OneHotEncoder,
)
from sklearn.preprocessing import TargetEncoder
from sklearn.decomposition import (
    PCA,
    SparsePCA
)
from sklearn.pipeline import (
    Pipeline,
    make_pipeline,
)
from sklearn.compose import (
    ColumnTransformer
)
from sklearn.model_selection import (
    train_test_split
)

In [4]:
PATH = os.getcwd()
PATH = PATH.split('/')[:-2]
PATH = ''.join([str(folder + '/') for folder in PATH])
print(PATH)

/Users/school/Documents/repositories/Datasets_EDA/src/Allstate Claims Severity/


In [5]:
with np.load(file=os.path.join(PATH, 'code', 'v2', 'Preprocessed_Pipeline_Data.npz')) as np_import:
    X_train= np_import['X_train']
    y_train= np_import['y_train']
    X_test= np_import['X_test']
    y_test= np_import['y_test']

## (A) Trees

In [6]:
from sklearn.metrics import (
    make_scorer,
    mean_absolute_error as mae,
    mean_squared_error as mse,
    r2_score as r2
)

In [7]:
from sklearn.tree import (
    DecisionTreeRegressor as DTR,
    plot_tree
)
from sklearn.ensemble import (
    RandomForestRegressor as RF,
    GradientBoostingRegressor as GBR
)

In [8]:
from sklearn.model_selection import (
    GridSearchCV,
)

In [9]:
scores = {
    'my_mae': make_scorer( # https://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html#sklearn.metrics.make_scorer
        score_func= mae,
    ),
    'my_mse': make_scorer(
        score_func= mse,
    ),
    'my_r2': make_scorer(
        score_func= r2,
    )
}

#### A.1: Decision Tree

In [10]:
# https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html#sklearn.tree.DecisionTreeRegressor
dtr = DTR(
    # criterion = "squared_error", # 'squared_error', 'friedman_mse', 'absolute_error', 'poisson'
    # splitter= "best", # 'best', 'random'
    # max_depth= None,
    # min_samples_split= 2,
    # min_samples_leaf= 1,
    # min_weight_fraction_leaf= 0,
    # max_features= 'sqrt', # 'sqrt', 'log2', None
    # random_state= 4095,
    # max_leaf_nodes= None,
    # min_impurity_decrease= 0,
    # ccp_alpha= 0
)

In [11]:
dtr_params = {
    'criterion': [ # Used to determine split on each node
        'squared_error', # DEFAULT: mse, minimize L2 loss
        'friedman_mse', # ?
        'absolute_error', # mae, minimize L1 loss
        #'poisson'
        ],
    'splitter': [ # how to split on each node
        'best', # DEFAULT 
        'random'
        ],
    # 'max_depth': [],
    # 'min_samples_split': [],
    # 'min_samples_leaf': [],
    # 'min_weight_fraction_leaf': [],
    'max_features': [ 
        'sqrt', # sqft(129) = 11
        'log2', # log_2(129) = 7
        20,
        40,
        60,
        80,
        100,
        120,
        ],
    'random_state': [
        4095
        ],
    # 'max_leaf_nodes': [],
    # 'min_impurity_decrease': [],
    # 'ccp_alpha': []
}

In [12]:
dtr_gs = GridSearchCV( # https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html#sklearn.model_selection.GridSearchCV
    estimator= dtr,
    param_grid= dtr_params,
    scoring= scores,
    n_jobs= -1,
    refit= False,
    cv= 3,
    verbose= 1,
    pre_dispatch= 8,
    # error_score: Float | str = ...,
    # return_train_score: bool = False
)

In [13]:
dtr_gs.fit(
    X= X_train,
    y= y_train,
)

Fitting 3 folds for each of 48 candidates, totalling 144 fits


GridSearchCV(cv=3, estimator=DecisionTreeRegressor(), n_jobs=-1,
             param_grid={'criterion': ['squared_error', 'friedman_mse',
                                       'absolute_error'],
                         'max_features': ['sqrt', 'log2', 20, 40, 60, 80, 100,
                                          120],
                         'random_state': [4095],
                         'splitter': ['best', 'random']},
             pre_dispatch=8, refit=False,
             scoring={'my_mae': make_scorer(mean_absolute_error, response_method='predict'),
                      'my_mse': make_scorer(mean_squared_error, response_method='predict'),
                      'my_r2': make_scorer(r2_score, response_method='predict')},
             verbose=1)

In [14]:
print(json.dumps(list(dtr_gs.cv_results_.keys()), indent=2))

[
  "mean_fit_time",
  "std_fit_time",
  "mean_score_time",
  "std_score_time",
  "param_criterion",
  "param_max_features",
  "param_random_state",
  "param_splitter",
  "params",
  "split0_test_my_mae",
  "split1_test_my_mae",
  "split2_test_my_mae",
  "mean_test_my_mae",
  "std_test_my_mae",
  "rank_test_my_mae",
  "split0_test_my_mse",
  "split1_test_my_mse",
  "split2_test_my_mse",
  "mean_test_my_mse",
  "std_test_my_mse",
  "rank_test_my_mse",
  "split0_test_my_r2",
  "split1_test_my_r2",
  "split2_test_my_r2",
  "mean_test_my_r2",
  "std_test_my_r2",
  "rank_test_my_r2"
]


In [15]:
dtr_mae_params= dtr_gs.cv_results_['params'][np.argmin(dtr_gs.cv_results_['mean_test_my_mae'])]
dtr_mse_params= dtr_gs.cv_results_['params'][np.argmin(dtr_gs.cv_results_['mean_test_my_mse'])]
dtr_r2_params= dtr_gs.cv_results_['params'][np.argmax(dtr_gs.cv_results_['mean_test_my_r2'])]

print(f"dtr_mae: {dtr_mae_params} | mae: {dtr_gs.cv_results_['mean_test_my_mae'][np.argmin(dtr_gs.cv_results_['mean_test_my_mae'])]:.2f}")
print(f"dtr_mse: {dtr_mse_params} | mse: {dtr_gs.cv_results_['mean_test_my_mse'][np.argmin(dtr_gs.cv_results_['mean_test_my_mse'])]:.2f}")
print(f"dtr_r2:  {dtr_r2_params}  | r2: {dtr_gs.cv_results_['mean_test_my_r2'][np.argmax(dtr_gs.cv_results_['mean_test_my_r2'])]:.2f}")

dtr_mae: {'criterion': 'friedman_mse', 'max_features': 120, 'random_state': 4095, 'splitter': 'random'} | mae: 1731.47
dtr_mse: {'criterion': 'friedman_mse', 'max_features': 120, 'random_state': 4095, 'splitter': 'random'} | mse: 8049443.94
dtr_r2:  {'criterion': 'friedman_mse', 'max_features': 120, 'random_state': 4095, 'splitter': 'random'}  | r2: 0.05


#### A.2: Random Forest

In [16]:
rf = RF( # https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html#sklearn.ensemble.RandomForestRegressor
    # n_estimators= 100,
    # criterion= "squared_error",
    # max_depth= None,
    # min_samples_split= 2,
    # min_samples_leaf= 1,
    # min_weight_fraction_leaf= 0,
    # max_features= 1,
    # max_leaf_nodes= None,
    # min_impurity_decrease= 0,
    # bootstrap= True,
    # oob_score= False,
    # n_jobs= None,
    # random_state= None,
    # verbose= 0,
    # warm_start= False,
    # ccp_alpha= 0,
    # max_samples= None
)

In [17]:
rf_params = {
    'n_estimators': [
        100,
        # 250,
        # 500,
        # 1000,
        ],
    'criterion': [
        'squared_error',
        'absolute_error',
        'friedman_mse'
        ],
    # 'max_depth': [],
    # 'min_samples_split': [],
    # 'min_samples_leaf': [],
    # 'min_weight_fraction_leaf': [],
    'max_features': [
        'sqrt',
        'log2',
        20,
        40,
        60,
        80,
        100,
        120,],
    # 'max_leaf_nodes': [],
    # 'min_impurity_decrease': [],
    # 'bootstrap': [],
    # 'oob_score': [],
    'n_jobs': [
        -1
        ],
    'random_state': [
        4095
        ],
    'verbose': [
        0
        ],
    'warm_start': [
        True,
        # False
        ],
    #'ccp_alpha': [],
    #'max_samples': []
}

In [18]:
rf_gs = GridSearchCV( # https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html#sklearn.model_selection.GridSearchCV
    estimator= rf,
    param_grid= rf_params,
    scoring= scores,
    n_jobs= -1,
    refit= False,
    cv= 3,
    verbose= 1,
    pre_dispatch= 8,
    # error_score: Float | str = ...,
    # return_train_score: bool = False
)

In [19]:
rf_gs.fit(
    X= X_train,
    y= y_train,
)

Fitting 3 folds for each of 24 candidates, totalling 72 fits


/opt/anaconda3/envs/matt_env/lib/python3.12/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


KeyboardInterrupt: 

In [ ]:
rf_mae_params= rf_gs.cv_results_['params'][np.argmin(rf_gs.cv_results_['mean_test_my_mae'])]
rf_mse_params= rf_gs.cv_results_['params'][np.argmin(rf_gs.cv_results_['mean_test_my_mse'])]
rf_r2_params= rf_gs.cv_results_['params'][np.argmax(rf_gs.cv_results_['mean_test_my_r2'])]

print(f"rf_mae: {rf_mae_params} | mae: {rf_gs.cv_results_['mean_test_my_mae'][np.argmin(rf_gs.cv_results_['mean_test_my_mae'])]:.2f}")
print(f"rf_mse: {rf_mse_params} | mse: {rf_gs.cv_results_['mean_test_my_mse'][np.argmin(rf_gs.cv_results_['mean_test_my_mse'])]:.2f}")
print(f"rf_r2:  {rf_r2_params}  | r2: {rf_gs.cv_results_['mean_test_my_r2'][np.argmax(rf_gs.cv_results_['mean_test_my_r2'])]:.2f}")

#### A.3: Gradient Boosting Regressor

In [ ]:
gbr = GBR( # https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingRegressor.html#sklearn.ensemble.GradientBoostingRegressor
    loss= "squared_error",
    learning_rate= 0.1,
    n_estimators= 100,
    subsample= 1,
    criterion= "friedman_mse",
    min_samples_split= 2,
    min_samples_leaf= 1,
    min_weight_fraction_leaf= 0,
    max_depth= 3,
    min_impurity_decrease= 0,
    init= None,
    random_state= None,
    max_features= None,
    alpha= 0.9,
    verbose= 0,
    max_leaf_nodes= None,
    warm_start= False,
    validation_fraction= 0.1,
    n_iter_no_change= None,
    tol= 0.0001,
    ccp_alpha= 0
)

In [ ]:
gbr_params = {
    'loss': [
        'squared_error',
        'absolute_error',
        'huber',
        # 'quantile'
        ],
    'learning_rate': [
        0.1,
        0.01,
        # 0.001
        ],
    'n_estimators': [
        100,
        250,
        # 500,
        ],
    #'subsample': [],
    'criterion': [
        'friedman_mse',
        'squared_error'
        ],
    # 'min_samples_split': [],
    # 'min_samples_leaf': [],
    # 'min_weight_fraction_leaf': [],
    # 'max_depth': [],
    # 'min_impurity_decrease': [],
    # 'init': [],
    'random_state': [
        4095
        ],
    'max_features': [
        'sqrt',
        'log2',
        20,
        40,
        60,
        80,
        100,
        120
        ],
    # 'alpha': [],
    'verbose': [
        0
        ],
    #'max_leaf_nodes': [],
    'warm_start': [
        True,
        #False
        ],
    'validation_fraction': [
        0.1,
        0.3
        ],
    'n_iter_no_change': [
        20
        ],
    #'tol': [],
    #'ccp_alpha': []
}

In [ ]:
gbr_gs = GridSearchCV( # https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html#sklearn.model_selection.GridSearchCV
    estimator= gbr,
    param_grid= gbr_params,
    scoring= scores,
    n_jobs= -1,
    refit= False,
    cv= 3,
    verbose= 1,
    pre_dispatch= 8,
    # error_score: Float | str = ...,
    # return_train_score: bool = False
)

In [ ]:
gbr_gs.fit(
    X= X_train,
    y= y_train,
)

Fitting 3 folds for each of 384 candidates, totalling 1152 fits


KeyboardInterrupt: 

In [ ]:
gbr_mae_params= gbr_gs.cv_results_['params'][np.argmin(gbr_gs.cv_results_['mean_test_my_mae'])]
gbr_mse_params= gbr_gs.cv_results_['params'][np.argmin(gbr_gs.cv_results_['mean_test_my_mse'])]
gbr_r2_params= gbr_gs.cv_results_['params'][np.argmax(gbr_gs.cv_results_['mean_test_my_r2'])]

print(f"rf_mae: {gbr_mae_params} | mae: {gbr_gs.cv_results_['mean_test_my_mae'][np.argmin(gbr_gs.cv_results_['mean_test_my_mae'])]:.2f}")
print(f"rf_mse: {gbr_mse_params} | mse: {gbr_gs.cv_results_['mean_test_my_mse'][np.argmin(gbr_gs.cv_results_['mean_test_my_mse'])]:.2f}")
print(f"rf_r2:  {gbr_r2_params}  | r2: {gbr_gs.cv_results_['mean_test_my_r2'][np.argmax(gbr_gs.cv_results_['mean_test_my_r2'])]:.2f}")